### Setup environment

In [ ]:
from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("250")')
import os

#Clone Repo
%cd /content
!git clone --recursive https://github.com/naver/dust3r
%cd /content/dust3r
!git checkout 88a43b939ebced21c069dd9df9e9e463324b7f5f

#Setup Environment
!pip install -r requirements.txt

if not os.path.exists('checkpoints'):
    #Download weights
    !mkdir -p checkpoints/
    !wget https://download.europe.naverlabs.com/ComputerVision/DUSt3R/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth -P checkpoints/

### Run **demo**

In [ ]:
%cd /content/dust3r

!sed -i 's/parser.add_argument("--weights", type=str, required=True, help="path to the model weights")/parser.add_argument("--weights", type=str, default='"'"'\/content\/dust3r\/checkpoints\/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth'"'"', help="path to the model weights")/' demo.py
!sed -i 's/demo.launch(share=False, server_name=server_name, server_port=server_port)/demo.launch(inline=True, debug=True)/' demo.py

from demo import *

#Dirty'n'quick trick to allow parse_args to work in colab cell without exploding
import sys; sys.argv = sys.argv[2:]

parser = get_args_parser()
args = parser.parse_args()

if args.tmp_dir is not None:
    tmp_path = args.tmp_dir
    os.makedirs(tmp_path, exist_ok=True)
    tempfile.tempdir = tmp_path

if args.server_name is not None:
    server_name = args.server_name
else:
    server_name = '0.0.0.0' if args.local_network else '127.0.0.1'

model = load_model(args.weights, args.device)
# dust3r will write the 3D model inside tmpdirname
with tempfile.TemporaryDirectory(suffix='dust3r_gradio_demo') as tmpdirname:
    print('Outputing stuff in', tmpdirname)
    main_demo(tmpdirname, model, args.device, args.image_size, server_name, args.server_port)

### OR if you want to use only CPU

In [ ]:

!git clone -b dev --recursive https://github.com/camenduru/dust3r


!pip install -q roma gradio einops trimesh
!pip install -q https://github.com/camenduru/wheels/releases/download/colab/curope-0.0.0-cp310-cp310-linux_x86_64.whl

!mkdir -p /content/dust3r/checkpoints
!wget https://huggingface.co/camenduru/dust3r/resolve/main/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth -P /content/dust3r/checkpoints

In [ ]:
!python /content/dust3r/demo.py --weights /content/dust3r/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth --tmp_dir /content/tmp